In [ ]:
# Imports & Configuration
import pandas as pd
from ollama import Client
from ddgs import DDGS
import csv
import time
import re
import os

MODEL_NAME = "gemini-3-flash-preview:cloud"
INPUT_FILE = "overbase_list.csv"
MISSING_DATA_FILE = "missing_data_rows.csv"
OUTPUT_FILE = "processed_data.csv"
BATCH_SIZE = 10

client = Client(host='http://localhost:11434')

print(f"Ollama initialized with Cloud Model: {MODEL_NAME}")

In [ ]:
# Helper Functions (Search, LLM, Regex)

COMPANY_DOMAIN_CACHE = {}

def search_web(query, max_results=3):
    """Searches DuckDuckGo and returns a summary string."""
    try:
        time.sleep(1) 
        results = DDGS().text(query, max_results=max_results)
        if not results: return ""
        return "\n".join([f"- {r['body']}" for r in results])
    except Exception as e:
        print(f"   [Search Error] {e}")
        return ""

def ask_ollama(prompt, context=""):
    """Queries the Ollama Cloud model."""
    full_prompt = f"""
    Context Data:
    {context}
    
    Instruction:
    {prompt}
    
    Output Rules:
    - Return ONLY the requested answer.
    - No markdown formatting (like **bold**), no filler text.
    """
    try:
        response = client.chat(model=MODEL_NAME, messages=[
            {'role': 'user', 'content': full_prompt},
        ])
        return response['message']['content'].strip()
    except Exception as e:
        return f"Error: {e}"

def extract_domain_with_regex(text):
    """Extracts a domain like 'google.com' from a text string using Regex."""
    # Pattern looks for domain-like structures (word.tld)
    pattern = r'\b(?:[a-zA-Z0-9](?:[a-zA-Z0-9-]{0,61}[a-zA-Z0-9])?\.)+[a-zA-Z]{2,6}\b'
    matches = re.findall(pattern, text.lower())
    
    if matches:
        return matches[0] # Return the first valid match
    return "n/a"

In [ ]:
# Load & Clean Data
def clean_and_load_data(filepath):
    valid_rows = []
    missing_data_rows = []
    INVALID_VALUES = {'', '-', '—', '–', 'n/a', 'nan', 'none'}

    try:
        with open(filepath, 'r', encoding='utf-8', errors='replace') as f:
            reader = csv.reader(f)
            header = next(reader, None)
            
            for row in reader:
                if not row: continue
                
                name, title, company = None, None, None
                if len(row) == 4:
                    name, title, company = row[0], row[1], row[2]
                elif len(row) >= 5:
                    name, title, company = row[0], row[1], row[3]
                else:
                    missing_data_rows.append(row)
                    continue

                name = name.strip() if name else ""
                title = title.strip() if title else ""
                company = company.strip() if company else ""

                is_title_bad = not title or title.lower() in INVALID_VALUES
                is_company_bad = not company or company.lower() in INVALID_VALUES

                if is_title_bad or is_company_bad:
                    missing_data_rows.append(row)
                else:
                    valid_rows.append({'Name': name, 'Title': title, 'Company': company})
                    
    except FileNotFoundError:
        print(f"Error: '{filepath}' not found.")
        return pd.DataFrame(), pd.DataFrame()

    return pd.DataFrame(valid_rows), pd.DataFrame(missing_data_rows)

print("📂 Loading data...")
df, df_missing = clean_and_load_data(INPUT_FILE)
df = df.drop_duplicates()

# Process batch trimming
if len(df) > BATCH_SIZE:
    df = df.head(BATCH_SIZE).copy()
    print(f"✂️  Dataset trimmed to {BATCH_SIZE} rows.")

# Split Names immediately
def split_name(full_name):
    parts = str(full_name).strip().split()
    first = parts[0]
    last = " ".join(parts[1:]) if len(parts) > 1 else ""
    return pd.Series([first, last])

if not df.empty:
    df[['First_Name', 'Last_Name']] = df['Name'].apply(split_name)

print(f"✅ Loaded {len(df)} rows.")

In [ ]:
# Filter Senior Executives

if not df.empty:
    print("🕵️  Step 1: Filtering Senior Executives...")
    
    def is_senior(title):
        prompt = f"Is '{title}' a Senior Executive role (C-Level, Director, Founder)? Answer strictly YES or NO."
        res = ask_ollama(prompt)
        return "YES" in res.upper()

    df['Is_Senior'] = df['Title'].apply(is_senior)
    
    # Filter DataFrame
    original_count = len(df)
    df = df[df['Is_Senior']].copy()
    print(f"   - Reduced from {original_count} to {len(df)} rows.")
else:
    print("⚠️ No data to process.")

In [ ]:
# Verify Employment & Get Domain
# We combine these to loop efficiently

def enrich_employment_and_domain(row):
    # --- A. Employment Verification ---
    # Query: "Person Company Title 2025 LinkedIn"
    search_query = f"{row['Name']} {row['Company']} {row['Title']} 2025 LinkedIn"
    work_context = search_web(search_query)
    
    verify_prompt = f"Based on the search results, is {row['Name']} still working at {row['Company']} as of 2025? Answer 'Yes', 'Likely No', or 'Uncertain'."
    still_working = ask_ollama(verify_prompt, work_context)

    # --- B. Domain Extraction (With Caching) ---
    company_name = row['Company']
    
    # Check Cache first
    if company_name in COMPANY_DOMAIN_CACHE:
        domain = COMPANY_DOMAIN_CACHE[company_name]
    else:
        # Search if not in cache
        domain_ctx = search_web(f"official website domain for {company_name}")
        # Use LLM + Regex to identify domain
        raw_domain = ask_ollama(f"Identify the main official website domain (e.g. apple.com) from these results.", domain_ctx)
        domain = extract_domain_with_regex(raw_domain)
        # Store in cache
        COMPANY_DOMAIN_CACHE[company_name] = domain

    return pd.Series([still_working, domain])

if not df.empty:
    print("🌍 Step 2 & 3: Verifying Employment & Extracting Domains...")
    df[['Employment_Status', 'Domain']] = df.apply(enrich_employment_and_domain, axis=1)
    print("   - Done.")

In [ ]:
# Step 4 - Generate Emails
# Using the domain and name to guess emails

def generate_emails(row):
    domain = row['Domain']
    if not domain or domain == 'n/a':
        return pd.Series(["N/A", "N/A"])

    # 1. Search for pattern
    pattern_ctx = search_web(f"email address format for {row['Company']} {domain} contact")
    
    # 2. Ask Ollama to generate
    prompt = f"""
    Based on the context (or standard corporate patterns if context is empty), generate the 2 most likely email addresses for:
    Name: {row['First_Name']} {row['Last_Name']}
    Company Domain: {domain}
    
    Strict Output Format: email1, email2
    (If only one is highly likely, output: email1, email1)
    """
    
    response = ask_ollama(prompt, pattern_ctx)
    
    # Clean up response to get strictly two columns
    try:
        # Remove any brackets, newlines, or labels
        clean_resp = response.replace('\n', ',').replace('Email 1:', '').replace('Email 2:', '')
        emails = [e.strip() for e in clean_resp.split(',') if '@' in e]
        
        email1 = emails[0] if len(emails) > 0 else "N/A"
        email2 = emails[1] if len(emails) > 1 else "N/A"
    except:
        email1, email2 = "Error", "Error"
        
    return pd.Series([email1, email2])

if not df.empty:
    print("📧 Step 4: Generating Emails...")
    df[['Email_1', 'Email_2']] = df.apply(generate_emails, axis=1)
    
    # Display Result
    display(df[['Name', 'Company', 'Employment_Status', 'Domain', 'Email_1', 'Email_2']].head())

In [ ]:
# Save Results
if not df.empty:
    final_cols = ['Name', 'Title', 'Company', 'Domain', 'Employment_Status', 'Email_1', 'Email_2']
    # Filter to ensure columns exist
    cols_to_save = [c for c in final_cols if c in df.columns]
    
    df[cols_to_save].to_csv(OUTPUT_FILE, index=False)
    print(f"💾 Saved {len(df)} validated leads to: {OUTPUT_FILE}")